In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import os
import warnings
warnings.filterwarnings("ignore")

######################
# METADATA
######################
sim_start = datetime.now()
sims=1000
input_folder = r'C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs'
input_file = 'inputs_v4_s1.xlsx'
path = os.path.join(input_folder, input_file)

######################
# READ IN DATA
######################
sales_ci=pd.read_excel(path, sheet_name='sales_ci')
unc_pr=pd.read_excel(path, sheet_name='unc_pr')
unc_pr_desc=pd.read_excel(path, sheet_name='unc_pr_desc')
unc_pri=pd.read_excel(path, sheet_name='unc_pri')
unc_pri_desc=pd.read_excel(path, sheet_name='unc_pri_desc')

######################
# PREP DATA
######################
sales_ci=sales_ci.drop_duplicates()
sales_ci=sales_ci.melt(id_vars=['product', 'region', 'units', 'indication'], 
        var_name="year", 
        value_name="sales")
sales_ci=sales_ci[(sales_ci['units'] =='¥')]

# Exclude indications that are not needed
exclude_ind = ['Adjustments', 'Central Adjustments', 'Total']
sales_ci=sales_ci[(~sales_ci['indication'].isin(exclude_ind))]

# Partition WW sales into 9 segments -> US, JP, CN, DE, FR, ES, IT, GB, WWex8
sales_ci_WW=sales_ci[(sales_ci['region']=='WW')]
ast8 = ['US','JP','CN', 'DE','FR','ES','IT','GB']
sales_ci_ast8=sales_ci[(sales_ci['region'].isin(ast8))]

sales_ci_WWex8=sales_ci_ast8.groupby(['product','units','indication','year']).sum().reset_index()
sales_ci_WWex8=sales_ci_WW.merge(sales_ci_WWex8, how='right', on=['product', 'units', 'indication', 'year'])
sales_ci_WWex8['sales']=sales_ci_WWex8['sales_x']-sales_ci_WWex8['sales_y']
sales_ci_WWex8['region']='WWex8'
sales_ci_WWex8 = sales_ci_WWex8[['product','region', 'units', 'indication', 'year', 'sales']]

# Generate new sales table
sales_ci_clean=pd.concat([sales_ci_ast8, sales_ci_WWex8])

# Add back non-strategic products
sales_ci_nsp=sales_ci[(sales_ci['indication']=='Non-Strategic')]
sales_ci_clean=pd.concat([sales_ci_clean, sales_ci_nsp])

# Create tag
sales_ci_clean['tag'] = sales_ci_clean['product'] + sales_ci_clean['region'] + sales_ci_clean['units'] + sales_ci_clean['indication'].astype(str)+ sales_ci_clean['year'].astype(str)

# Validation
print(sum(sales_ci_clean[(sales_ci_clean['year']==2025)]['sales']))  #18491
print(sum(sales_ci_clean[(sales_ci_clean['product']=='xtandi') & (sales_ci_clean['year']==2025)]['sales']))  #7309
print(sum(sales_ci_clean[(sales_ci_clean['product']=='mirabegron') & (sales_ci_clean['year']==2025)]['sales']))  #1223
print(sales_ci_clean['tag'].nunique()) # 920

# Save new sales data
sales_ci_clean.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\sales_ci_clean.csv")

18491.327050511707
7309.662391952652
1223.22
920


In [2]:
#############
# PHASE 1: PTRS
#############
# Create output df
output_df = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50', 'sales_RA_75', 'sales_RA_90'])

# Add PTRS values to sales data
sales_ci_unc = sales_ci_clean.merge(unc_pri, how='right', on=['product','region', 'units', 'indication', 'year'])

# Loop through rows
for index, row in sales_ci_unc.iterrows():
# for index, row in sales_ci_unc.iloc[29:30].iterrows():
    n=sims
    sales_ra = []
    for i in range(n):
        
        # Get sales (value)
        sales = row.values[5]
        
        # Get PTRS Uncertainty Probability (value)
        unc_ptrs_prob=np.random.binomial(size=1, n=1, p=row.values[7])
        
        # Get Uncertainty Value (value) for others
        unc1=row.values[8]
        unc2=row.values[9]
        
        if ((unc_ptrs_prob == 1) and ((~np.isnan(unc1)) or (~np.isnan(unc2)) )):
            m=sims
            for j in range(m):

                # Get Uncertainty Probability (value) for others
                unc1_prob=np.random.randint(1,4)
                
                if ((unc1_prob == 1)):
                    sales=unc1
                
                if ((unc1_prob == 2)):
                    sales=unc2
        
        # Generate RA sales
        unc_all=float(sales)*float(unc_ptrs_prob)
        sales_ra.append(unc_all)
        
    # Get Product Worldwide sales by year (series)
    prod_ww_sales = sales_ci_unc.iloc[index]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_RA_mean'] = np.mean(sales_ra)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

# Save outputs
output_df = output_df[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50','sales_RA_75', 'sales_RA_90', 'sales_RA_mean']]

sales_exPTRS=sales_ci_clean[~(sales_ci_clean['tag'].isin(output_df['tag']))]
sales_exPTRS['sales_RA_10']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_25']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_50']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_75']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_90']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_mean']=sales_exPTRS['sales']
output_df=pd.concat([output_df, sales_exPTRS])

# Validation
print(sum(output_df[(output_df['year']==2025)]['sales']))  #18491
print(sum(output_df[(output_df['product']=='xtandi') & (output_df['year']==2025)]['sales'])) #7309
print(sum(output_df[(output_df['product']=='mirabegron') & (output_df['year']==2025)]['sales']))  #1223
print(output_df['tag'].nunique()) # 920

output_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase1.csv")

18491.327050511714
7309.662391952652
1223.22
920


In [3]:
#############
# PHASE 2: COMMERCIAL UNCERTAINTY
#############
# Convert PTRS results to sales df for next phase (Commercial Uncertainty)
sales_ptrs=output_df[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_mean']]
sales_ptrs.columns = ['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_p1_RA']

# Sum sales at the WW level
sales_ptrs_ww=sales_ptrs.groupby(['product','units','indication','year']).sum().reset_index()
sales_ptrs_ww['region']='Total'
sales_ptrs=pd.concat([sales_ptrs_ww, sales_ptrs])

# Sum sales at the Product level, as commercial uncertainty is applied at a higher granularity
sales_ptrs_prod=sales_ptrs.groupby(['product','units','region','year']).sum().reset_index()
sales_ptrs_prod['indication']='Total'
sales_ptrs=pd.concat([sales_ptrs_prod, sales_ptrs])

# Validation
print(sum(sales_ptrs[(sales_ptrs['year']==2025)& (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #18491
print(sum(sales_ptrs[(sales_ptrs['product']=='xtandi') & (sales_ptrs['year']==2025) & (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #7309
print(sum(sales_ptrs[(sales_ptrs['product']=='mirabegron') & (sales_ptrs['year']==2025) & (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #1223
print(sales_ptrs[(sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['tag'].nunique()) # 920

18491.327050511714
7309.662391952652
1223.22
920


In [4]:
######################
# CREATE INPUT & OUTPUT DF
######################
output_df2 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25','sales_RA_50','sales_RA_75', 'sales_RA_90'])

# Create input df
sales_p2a=sales_ptrs[(sales_ptrs['region']=='Total') & (sales_ptrs['indication']=='Total')]
sales_p2b=sales_ptrs[(sales_ptrs['region']!='Total') & (sales_ptrs['indication'].isin(['Total']))]

# Validation
print(sum(sales_p2a[(sales_p2a['year']==2025)]['sales']))  #18491
print(sum(sales_p2a[(sales_p2a['product']=='xtandi') & (sales_p2a['year']==2025)]['sales']))  #7309
print(sum(sales_p2a[(sales_p2a['product']=='mirabegron') & (sales_p2a['year']==2025)]['sales']))  #1223

print(sum(sales_p2b[(sales_p2b['year']==2025)]['sales']))  #18491
print(sum(sales_p2b[(sales_p2b['product']=='xtandi') & (sales_p2b['year']==2025)]['sales']))  #7309
print(sum(sales_p2b[(sales_p2b['product']=='mirabegron') & (sales_p2b['year']==2025)]['sales']))  #1223

18491.32705051171
7309.662391952652
1223.22
18491.3270505117
7309.66239195265
1223.2200000000003


In [5]:
#############
# XTANDI
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xtandi'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc1_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc1')]['probability'], 1, 1)
            unc2_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc2')]['probability'], 1, 1)
            unc3_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc3')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc1=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc1']
            unc2=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc2']
            unc3=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc3']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc1*unc1_prob) + float(unc2*unc2_prob) + float(unc3*unc3_prob)
            sales_ra.append(unc_all)
            
        # Get Product Region sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
5,0,xtandi,IT,¥,Total,2022,127.421278,124.371116,125.588358,127.385671,129.364213,130.407127,127.421278,127.442645
6,0,xtandi,ES,¥,Total,2022,99.871451,97.580158,98.455732,99.967627,101.431933,102.230563,99.871451,99.954603
7,0,xtandi,GB,¥,Total,2022,116.406916,113.492165,114.614715,116.274560,118.201907,119.313690,116.406916,116.391594
8,0,xtandi,WWex8,¥,Total,2022,1087.663909,1061.677897,1071.545533,1086.932178,1103.879547,1112.792143,1087.663909,1087.377210
9,0,xtandi,US,¥,Total,2023,3892.422908,3733.927835,3771.106693,3832.533548,3894.785594,3938.585812,3770.159986,3834.177150


In [6]:
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_75']))

26990.006769473104
26266.31832125493
26682.769995676194
27105.42148545019


In [7]:
#############
# XOSPATA
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xospata'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]


for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc4_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc4')]['probability'], 1, 1)
            unc5_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc5')]['probability'], 1, 1)
            unc6_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc6')]['probability'], 1, 1)

            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get relevant sales values for uncertainties
            unc4_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            unc5_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            unc6_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            
            # Get Uncertainty Quant by year (series)
            unc4=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc4'])*float(unc4_sales)
            unc5=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc5'])*float(unc5_sales)
            unc6=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc6'])*float(unc6_sales)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc4*unc4_prob) + float(unc5*unc5_prob) + float(unc6*unc6_prob)
            sales_ra.append(unc_all)
            
        # Get Product Region sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,xospata,FR,¥,Total,2025,10.287643,11.072082,11.285015,11.597613,11.851808,12.064579,12.143998,11.576652
68,0,xospata,IT,¥,Total,2025,41.269885,43.667515,44.486371,45.810162,47.001487,47.699196,48.027963,45.744709
69,0,xospata,ES,¥,Total,2025,9.511894,9.098342,9.259928,9.516359,9.770418,9.930526,10.029448,9.514936
70,0,xospata,GB,¥,Total,2025,13.035730,13.356702,13.632401,13.984152,14.397499,14.642116,14.728095,13.998667


In [8]:
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_75']))   

2353.661500188844
2186.765717490872
2244.931881585494
2302.062005786622


In [9]:
#############
# PADCEV
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'padcev'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc7_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc7')]['probability'], 1, 1)
            unc8_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc8')]['probability'], 1, 1)
            unc9_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc9')]['probability'], 1, 1)
            unc10_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc10')]['probability'], 1, 1)
            unc11_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc11')]['probability'], 1, 1)
            unc12_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc12')]['probability'], 1, 1)
            unc13_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc13')]['probability'], 1, 1)
            unc14_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc14')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc7=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc7']
            unc8=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc8']
            unc9=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc9']
            unc10=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc10']
            unc11=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc11']
            unc12=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc12']
            unc13=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc13']
            unc14=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc14']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc7*unc7_prob) + float(unc8*unc8_prob) + float(unc9*unc9_prob) + float(unc10*unc10_prob) + float(unc11*unc11_prob) + float(unc12*unc12_prob) + float(unc13*unc13_prob) + float(unc14*unc14_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0,padcev,FR,¥,Total,2025,30.210385,10.598419,15.005259,20.544861,25.381348,28.381053,29.781748,20.062257
104,0,padcev,IT,¥,Total,2025,29.306934,25.816143,27.093210,29.392125,31.585183,32.797907,29.291693,29.381472
105,0,padcev,ES,¥,Total,2025,13.922966,12.200615,12.850161,13.950974,14.955378,15.558701,13.892976,13.912928
106,0,padcev,GB,¥,Total,2025,0.492994,0.433040,0.453234,0.491495,0.528317,0.551093,0.492994,0.491190


In [10]:
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_75']))

4084.8039713788817
3730.4630279745234
4053.191500764783
4394.066362130707


In [11]:
#############
# EVRENZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'evrenzo'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc15_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc15')]['probability'], 1, 1)
            unc16_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc16')]['probability'], 1, 1)
            unc17_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc17')]['probability'], 1, 1)
            unc18_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc18')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc15=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc15']
            unc16=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc16']
            unc17=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc17']
            unc18=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc18']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc15*unc15_prob) + float(unc16*unc16_prob) + float(unc17*unc17_prob) + float(unc18*unc18_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0,evrenzo,FR,¥,Total,2025,46.025024,38.922268,41.114194,44.586140,47.926533,50.048458,46.025024,44.584141
140,0,evrenzo,IT,¥,Total,2025,72.868764,44.528760,49.961113,56.239696,63.328967,69.259170,72.868764,56.655800
141,0,evrenzo,ES,¥,Total,2025,14.194413,10.396776,11.299660,12.353039,13.568353,14.474334,14.194413,12.411440
142,0,evrenzo,GB,¥,Total,2025,17.100499,15.139610,15.865093,17.143524,18.410967,19.122924,17.100499,17.140564


In [12]:
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_75']))

1113.6799883146336
1012.8798434416126
1099.1447450192882
1186.1386372552847


In [13]:
#############
# ZOLBE
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'zolbe'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []
        
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0,zolbe,FR,¥,Total,2025,2.234815,1.722408,1.811448,1.941010,2.093841,2.186939,1.957698,1.950142
176,0,zolbe,IT,¥,Total,2025,3.637843,2.770832,2.911525,3.148198,3.389413,3.536079,3.146734,3.151435
177,0,zolbe,ES,¥,Total,2025,1.740246,1.356199,1.436118,1.555412,1.664853,1.726609,1.540118,1.549429
178,0,zolbe,GB,¥,Total,2025,2.423964,1.856568,1.943400,2.106794,2.271494,2.363948,2.118545,2.106955


In [14]:
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_75']))  

475.20551843110303
383.47842399997086
414.0533418421099
445.9523980646547


In [15]:
#############
# FEZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'fezo'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    # Get Fezo US values, to create proportion
    fezo_base=sales_ptrs[(sales_ptrs['product']==product) & (sales_ptrs['region']=='US') & (sales_ptrs['year']==year)& (sales_ptrs['indication']=='Total')]['sales_p1_RA']
    fezo_10 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc19'])
    fezo_25 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc20'])
    fezo_50 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc21'])
    fezo_75 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc22'])
    fezo_90 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc23'])
    
    for region in regions:
        # Get Product Worldwide sales by year (series) 
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year)]
        prod_ww_sales['sales_RA_10'] = float(fezo_10/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_25'] = float(fezo_25/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_50'] = float(fezo_50/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_75'] = float(fezo_75/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_90'] = float(fezo_90/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
    
        prod_ww_sales['sales_p2_RA'] = prod_ww_sales['sales_RA_50']

        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0,fezo,FR,¥,Total,2025,14.440805,7.616331,9.536032,12.262236,16.005086,19.611625,14.151989,12.262236
212,0,fezo,IT,¥,Total,2025,12.874491,6.824873,8.545088,10.987995,14.341904,17.573667,12.681373,10.987995
213,0,fezo,ES,¥,Total,2025,5.675226,2.996267,3.751478,4.823967,6.296406,7.715220,5.567397,4.823967
214,0,fezo,GB,¥,Total,2025,17.030000,9.018578,11.291717,14.519843,18.951791,23.222333,16.757520,14.519843


In [16]:
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_75']))   

5336.288938830225
3531.082767199254
4540.405920501046
5926.552859191375


In [17]:
#############
# MIRABEGRON
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'mirabegron'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for region in regions:
    n=sims
    sales_ra22 = []
    sales_ra23 = []
    sales_ra24 = []
    sales_ra25 = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Get High,Base,Low Uncertainty
        unc_hbl_prob=np.random.randint(1,4)
        # unc_hbl_prob=1

        # Generate RA sales - High
        if (unc_hbl_prob==1):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc24'])*float(1+unc_base_prob)

        # Generate RA sales - Med
        if (unc_hbl_prob==2):
            sales22=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2022) ]['sales'])*float(1+unc_base_prob)
            sales23=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2023) ]['sales'])*float(1+unc_base_prob)
            sales24=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2024) ]['sales'])*float(1+unc_base_prob)
            sales25=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2025) ]['sales'])*float(1+unc_base_prob)
        
        # Generate RA sales - Low
        if (unc_hbl_prob==3):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc25'])*float(1+unc_base_prob)
        
        sales_ra22.append(sales22)
        sales_ra23.append(sales23)
        sales_ra24.append(sales24)
        sales_ra25.append(sales25)
 
     # Get Product, Region sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2022) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra22, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra22, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra22, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra22, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra22, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra22)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2023) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra23, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra23, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra23, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra23, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra23, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra23)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2024) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra24, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra24, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra24, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra24, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra24, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra24)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2025) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra25, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra25, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra25, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra25, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra25, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra25)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,mirabegron,GB,¥,Total,2025,69.956730,67.226145,68.266213,69.918832,71.614344,72.773063,69.956730,69.964704
248,0,mirabegron,WWex8,¥,Total,2022,306.367270,294.191602,298.546883,306.623084,313.707564,318.526750,306.367270,306.311578
249,0,mirabegron,WWex8,¥,Total,2023,303.232806,287.687335,292.258514,300.042354,307.255940,311.216992,303.232806,299.749922
250,0,mirabegron,WWex8,¥,Total,2024,290.662682,275.551034,279.940672,287.505427,294.333984,298.103869,290.662682,287.182535


In [18]:
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_75']))  

6512.1
5136.198453561739
6487.559809973777
7126.623494384463


In [19]:
#############
# TACROLIMUS
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'tacrolimus'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=sims
        sales_ra = []

        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)
            
            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)
            
        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,tacrolimus,FR,¥,Total,2025,123.608011,118.791516,120.482080,123.459100,126.579536,128.320648,123.608011,123.496484
284,0,tacrolimus,IT,¥,Total,2025,36.814557,35.326294,35.845688,36.827531,37.787870,38.355094,36.814557,36.828946
285,0,tacrolimus,ES,¥,Total,2025,59.892065,57.536934,58.390562,59.925880,61.388235,62.289161,59.892065,59.906675
286,0,tacrolimus,GB,¥,Total,2025,54.303272,52.148532,52.916666,54.248645,55.656060,56.490033,54.303272,54.286119


In [20]:
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_75']))  

6736.2300000000005
6566.770376154949
6740.588515082217
6906.481767951586


In [21]:
#############
# LEXISCAN
#############
years = [2022, 2023, 2024, 2025]
product = 'lexiscan'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    n=sims
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)& (unc_pr['region'] =='US')]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['year']==year) ]
        unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)

    
    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0,tacrolimus,WWex8,¥,Total,2025,706.677430,678.922659,689.078164,707.370630,725.315986,735.503846,706.677430,707.176200
288,0,lexiscan,WW,¥,Total,2022,759.733936,728.621192,742.244398,760.754030,779.369877,790.332810,759.733936,760.342453
289,0,lexiscan,WW,¥,Total,2023,379.866968,364.121724,369.751090,380.282845,389.708499,394.979660,379.866968,379.894931
290,0,lexiscan,WW,¥,Total,2024,18.993348,18.214264,18.502851,18.977323,19.427625,19.759537,18.993348,18.981821


In [22]:
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_75']))

1174.7385985710891
1146.2290044657182
1176.1455275691978
1205.035419756525


In [23]:
#############
# OTHERS
#############
years = [2022, 2023, 2024, 2025]
region = 'WW'
other_products = ['evenity','cresemba','tamsulosin','suglat','ambisome','vesicare','mycamine','sujanu','symraf','cimzia','blincyto','repatha','linzess',
'vesomni','myslee/stilnox','irribow','gonax','feburic','asamax','josamycin','non prod','istodax','other merchandise products','allelock','tr','acr','blz','bonoteo','seroquel','geninax','col','inf-v','kiklin',
'ofa','regnite','srs','st','dificlir','p_aco - acofide','other astellas products']

for product in other_products:
    # Clear output for rerun
    output_df2=output_df2[(output_df2['product']!=product)]
    print(product)
    for year in years:
        n=sims
        sales_ra = []
    
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    

        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

evenity
cresemba
tamsulosin
suglat
ambisome
vesicare
mycamine
sujanu
symraf
cimzia
blincyto
repatha
linzess
vesomni
myslee/stilnox
irribow
gonax
feburic
asamax
josamycin
non prod
istodax
other merchandise products
allelock
tr
acr
blz
bonoteo
seroquel
geninax
col
inf-v
kiklin
ofa
regnite
srs
st
dificlir
p_aco - acofide
other astellas products


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3363.517488,3396.169499,3445.838458,3503.591197,3532.318196,3403.336080,3448.686574
1,0,xtandi,JP,¥,Total,2022,438.632820,428.535157,432.026165,439.149107,445.367752,449.485721,438.632820,438.864718
2,0,xtandi,CN,¥,Total,2022,67.308338,65.611233,66.212232,67.293449,68.215592,68.871971,67.308338,67.245838
3,0,xtandi,DE,¥,Total,2022,518.145979,505.840696,509.679307,518.808309,525.963711,530.572195,518.132744,518.235697
4,0,xtandi,FR,¥,Total,2022,260.011082,253.832133,255.926857,259.690026,263.888795,266.331858,260.011082,259.897505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,0,p_aco - acofide,WW,¥,Total,2025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
448,0,other astellas products,WW,¥,Total,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
449,0,other astellas products,WW,¥,Total,2023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
450,0,other astellas products,WW,¥,Total,2024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_75']))

9797.389256678245
9305.729615310285
9798.557403565044
10289.517377503122


In [25]:
# Save output of Phase 2
output_df2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase2.csv")

# Add WW sales to output
output_df3=output_df2
output_df3=output_df3.groupby(['product','units','indication','year']).sum().reset_index()
output_df3['region']='ALL'
output_df3 = output_df2.append(output_df3, ignore_index=True)

output_df3.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase2_agg.csv")

In [26]:
######################
# METADATA
######################
sim_end = datetime.now()
td =  sim_end - sim_start
td_mins = int(round(td.total_seconds() / 60))
metatdata = {'sim_start': sim_start, 'sim_end': sim_end, 'sims_run': sims, 'sim_time': td_mins}
metatdata_df=pd.DataFrame.from_dict(metatdata, orient='index')
metatdata_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\metatdata.csv")